In [1]:
from src.guess.guess import ConfigType, Guess, Config
from src.domain.datasets.UnixCommandDataset import UnixCommandDataset  # Register dataset
from src.domain.datasets.ProjectEulerDataset import ProjectEulerDataset
from src.domain.datasets.BringUpDataset import BringUpDataset
from src.domain.datasets.HumanEvalDataset import HumanEvalDataset

prediction = None

config = Config(ConfigType.QWEN_X862ARM64.get_path())
guess = Guess(config=config)

data_loader = guess.data_loader.iter()

data_loader.__next__()

test_instance = data_loader.__next__()

result = guess.model.predict(test_instance, guess.inference_cfg)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-11 23:14:16,006 - src.domain.models.QwenModel - INFO - Initializing QwenModel with ahmedheakl/ex19_qwen2.5-1.5b-1m-stack-16kcw on device mps


Loading Qwen model: ahmedheakl/ex19_qwen2.5-1.5b-1m-stack-16kcw


2025-05-11 23:14:18,943 - QwenModel - INFO - Initialized QwenModel on device: mps:0
2025-05-11 23:14:18,944 - src.domain.models.QwenModel - INFO - Model initialization completed in 2.94 seconds
2025-05-11 23:14:18,947 - src.helpers.dataset - INFO - Loaded 31 instances
2025-05-11 23:14:18,947 - src.helpers.data_loader - INFO - Loaded 31 samples from dataset
2025-05-11 23:14:18,957 - src.domain.models.QwenModel - INFO - Starting inference with temperature=0.7, max_length=4096
2025-05-11 23:14:18,957 - src.domain.models.QwenModel - INFO - Input shape: torch.Size([1, 654])
`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Source JSONL: data/processed/X86/BringUp_x86.jsonl
Target JSONL: data/processed/ARM64/BringUp_arm.jsonl
Loading file: data/processed/X86/BringUp_x86.jsonl
Loaded 31 entries from data/processed/X86/BringUp_x86.jsonl
Loading file: data/processed/ARM64/BringUp_arm.jsonl
Loaded 31 entries from data/processed/ARM64/BringUp_arm.jsonl
Source entries: 31
Target entries: 31
Creating instance 0 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output'])
Creating instance 1 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output'])
Creating instance 2 with source key x86 and target key arm
Source entry keys: dict_keys(['source', 'x86', 'x86_output'])
Target entry keys: dict_keys(['source', 'arm', 'arm_output'])
Creating instance 3 with source key x86 and target key arm
Source entry keys: dict_

In [6]:
print(len(result.alignments))
print(len(result.confidence))
print(len(result.source[0]))
print(len(result.pred[0]))

1394
1394
654
1394


In [7]:
from src.sketch.sketch import Sketch

sketch = Sketch(config, guess.model)

In [18]:
from src.sketch.fixes import fix_duplicate_sections, fix_missing_sections

In [19]:
fixed_pred = fix_missing_sections(sketch, result)

Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
Missing section: LBB1_1
